#### Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [2]:
os.chdir("/kaggle/input/dataset-homework2")
!ls 

### Set seed for reproducibility

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Load the dataset

In [4]:
dataset = pd.read_csv('Training.csv')

dataset = pd.concat([dataset[:67103], dataset[67199:]], ignore_index=True)

dataset = dataset.astype('float32')
print(dataset.shape)
dataset.head()

In [5]:
os.chdir("/kaggle/working/")

### Exploration Data Analysis (EDA)

In [6]:
dataset.info()

### Sequential Train-Test split and normalization

In [7]:
X_train_raw = dataset.iloc[-42000:] # Take only the last 42000 points 
print(X_train_raw.shape)

# Normalization
X_min = X_train_raw.min()
X_max = X_train_raw.max()
X_train_raw = (X_train_raw-X_min)/(X_max-X_min)

plt.figure(figsize=(17,5))
plt.plot(X_train_raw.Sponginess, label='Train')
plt.title('Train-Validation Split')
plt.legend()
plt.show()

### Set Window, stride and telescope

In [8]:
window = 1000
stride = 20
target_labels = dataset.columns
telescope = 100

### Create train and validation sequences

In [9]:
def build_sequences(df, target_labels, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [10]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape

### Separate the inputs in 7 different inputs

Since the model is Multi-Input, we need to separate the 7 columns and put each of them in a list. The output is still 1, so we only do this for the X, not for the y.

In [11]:
def separate_features(X_train, X_val=None, n_separations=7):
    X_train_list = []
    X_val_list = []
    
    for n in range(n_separations):
        X_train_list.append(X_train[:,:, n])
        if X_val != None:
            X_val_list.append(X_val[:,:, n])
    if X_val == None:
        return X_train_list
    else:
        return X_train_list, X_val_list
            

In [12]:
X_train_list = separate_features(X_train, n_separations = 7)     

### Set hyperparameters (batch_size, epochs...)

In [13]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 128
epochs = 100

In [14]:
print(input_shape)
print(output_shape)

### Build the Multi-input model

In questo caso, n_filters decide quanti filtri in parallelo mettere PER OGNI INPUT. Quindi se metto n_filters=2 e passo i dati come 7 input separati (1 per colonna) e lascio depth=1, avrò 2 filtri per ogni input, per un totale di 14 filtri. "input_shape" deve rappresentare la shape di OGNI input, quindi deve essere uguale per tutti gli input (per ora, potrei cambiarla in futuro).

In [15]:
def create_multiInput_convnet(input_shape, output_shape, n_filters, depth=1, verbose=0):
    
    # Create all the input layers
    input_layer1 = tfkl.Input(shape=input_shape, name="Input1") 
    input_layer2 = tfkl.Input(shape=input_shape, name="Input2")
    input_layer3 = tfkl.Input(shape=input_shape, name="Input3")
    input_layer4 = tfkl.Input(shape=input_shape, name="Input4")
    input_layer5 = tfkl.Input(shape=input_shape, name="Input5")
    input_layer6 = tfkl.Input(shape=input_shape, name="Input6")
    input_layer7 = tfkl.Input(shape=input_shape, name="Input7")
    
    # Create a list of Input layers
    input_list = [input_layer1, input_layer2, input_layer3, input_layer4, input_layer5, input_layer6, input_layer7]
    
    # Create an empty list: I will append the final layer of each input there
    output_list = []
    
    #####################################
    #### LOOP THROUGH ALL THE INPUTS ####
    #####################################
    ### All the tower creation that comes next has to be done separatly for all the inputs:
    ### in order to do so we can loop
    for input_layer in input_list: 
          
        residual_layers = []  # I initialize an empty list for the residual connections
        
        ########################
        ## Create first tower ##
        ########################
        tower_n1_d1 = tfkl.Conv1D(1, kernel_size=1, padding='same', activation='relu')(input_layer)
        tower_n1_d1 = tfkl.MaxPool1D(pool_size=2, strides=2, padding='same')(tower_n1_d1)
        #residual_layers.append(tower_n1_d1)

        ###################################################
        #### Create deeper blocks of the first tower ######
        ###################################################
        for level in range(2, depth+1):  # level 1 already created, starting from 2
            if level == 2:
                tower_n1 = tfkl.Conv1D(1, kernel_size=1, padding='same', activation='relu')(tower_n1_d1)
                tower_n1 = tfkl.MaxPool1D(pool_size=2, strides=2, padding='same')(tower_n1)
                residual_layers.append(tower_n1)  # This is the residuals we ended up using in the best model
            else:  # Keep adding blocks until we reach the last "depth" level                                              
                tower_n1 = tfkl.Conv1D(1, kernel_size=1, padding='same', activation='relu')(tower_n1)
                tower_n1 = tfkl.MaxPool1D(pool_size=2, strides=2, padding='same')(tower_n1)
                #residual_layers.append(tower_n1)


        ###############################
        ### Create the other towers ###
        ###############################
        # We created the first "tower", the one with filter of size 1. Now we do the one 
        # with filter size equal to 2, then 3 etc. until the last one decided by n_filters
        # The loop starts from 2 (1 done before the loop) and end at n_filter+1 (range don't take last elem)
        # The rest of the code is basically identical to the one above, with 
        # different variable names
        for kernel_window in range(2, n_filters+1):  
            tower_d1 = tfkl.Conv1D(1, kernel_size=kernel_window, padding='same', activation='relu')(input_layer)
            tower_d1 = tfkl.MaxPool1D(pool_size=2, strides=2, padding='same')(tower_d1)
            #residual_layers.append(tower_d1)

          ####################################################
          #### Create deeper blocks of the other towers ######
          ####################################################
            for level in range(2, depth+1):  # level 1 already created, starting from 2
                if level == 2:
                    tower = tfkl.Conv1D(1, kernel_size=kernel_window, padding='same', activation='relu')(tower_d1)
                    tower = tfkl.MaxPool1D(pool_size=2, strides=2, padding='same')(tower)
                    residual_layers.append(tower)
                else:
                    tower = tfkl.Conv1D(1, kernel_size=kernel_window, padding='same', activation='relu')(tower)
                    tower = tfkl.MaxPool1D(pool_size=2, strides=2, padding='same')(tower)
                    #residual_layers.append(tower)


            ###############################
            ## CONCATENATE CURRENT TOWER ##
            ###############################
            # After finishing creating a "tower" with a certain filter size, 
            # before going on to the next filter size, concatenate the current 
            # one in order to keep it in the layer "merged".            
            if kernel_window == 2:  # it means first iteration, create merged
                if verbose>0:
                    print("tower_n1 shape:", tower_n1.shape,"\t tower shape:", tower.shape)
                merged = tfkl.concatenate([tower_n1, tower], axis=1)
            else:  # we already created merged, now append it the new tower
                if verbose>0:
                    print("merged shape:", merged.shape,"\t tower shape:", tower.shape)
                merged = tfkl.concatenate([merged, tower], axis=1)           
      
        ### Now, before moving to the next input, we want to append part  ###
        # of the input itself to the output, basically skipping all 
        # the feature extraction part.
        skipped_input = input_layer[:, -100:, :]
        merged = tfkl.concatenate([merged, skipped_input], axis=1)
        
        # Now we have to concatenate the "residuals" that we saved in the residual_layers list
        for residual in residual_layers:
            merged = tfkl.concatenate([merged, residual], axis=1)           
    
        # When we are done merging the filters, flatten them
        merged = tfkl.Flatten()(merged)
        
        # Now we can append this layer to our list of outputs:
        output_list.append(merged)
        
        # Now the cycle will start over from another input etc. until all inputs are processed
  
    
    # Now we have a list with all our output layers created. In order to use them we need to
    # concatenate them. I'm doing it separately (first putting them in a list, then concatenating them)
    # just because this way the code if easier for me to debug.
    output_merged =  tfkl.concatenate([output_list[0], output_list[1]], axis=1)
    for elem in output_list[2:]:
        output_merged = tfkl.concatenate([output_merged, elem], axis=1)
    
    dropout = tfkl.Dropout(0.2)(output_merged)  # Dropout added to mitigate overfitting
    out = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='hard_sigmoid', name="Dense")(dropout)  #hard_sigmoid outperformed all 
    out = tfkl.Reshape((output_shape[-2],output_shape[-1]), name="Reshape")(out)                           #the other activation functions

    model = tfk.Model(input_list, out, name='MultiInput_model')
    
    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])
    return model

In [16]:
model = create_multiInput_convnet((input_shape[0], 1), output_shape, 8, 4)  # input_shape[0] is basically the windowsize in this case
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [17]:
X_train_list[0].shape, y_train.shape

### Train

In [18]:
# Train the model
history = model.fit(
    x = {"Input1": X_train_list[0], "Input2": X_train_list[1], 
         "Input3": X_train_list[2], "Input4": X_train_list[3], 
         "Input5": X_train_list[4], "Input6": X_train_list[5], "Input7": X_train_list[6]},
    y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=.1,
    callbacks = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True)
).history

### Save the model

In [19]:
model.save('G26_hard_sigmoid')